In [1]:
# Homework 2: Word Similarity

Student Name:Shenglu Zhang

Student ID:795682

Python version used:3.5

## General info

<b>Due date</b>: 1pm, Sunday April 1st

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 5% of mark for class

<b>Overview</b>: In this homework, you'll be quantifying the similarity between pairs of words using the structure of WordNet and word co-occurrence in the Brown corpus, using PMI, LSA, and word2vec. You will quantify how well these methods work by comparing to a carefully filtered human annotated gold-standard.

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> It is recommended to use Python 2 but we accept Python 3 solutions</b>. Make sure you state which version you used in the beggining of this notebook.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Extra credit</b>: Each homework has a task which is optional with respect to getting full marks on the assignment, but that can be used to offset any points lost on this or any other homework assignment (but not the final project or the exam). We recommend you skip over this step on your first pass, and come back if you have time: the amount of effort required to receive full marks (1 point) on an extra credit question will be substantially more than earning the same amount of credit on other parts of the homework.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.


<b>Instructions</b>: For this homework we will be comparing our methods against a popular dataset of word similarities called Similarity-353. You need to first obtain this data set, which can be downloaded <a href="http://www.cs.technion.ac.il/~gabr/resources/data/wordsim353/wordsim353.zip">here</a>. The file we will be using is called *combined.tab*. Except for the header (which should be stripped out), the file is tab formated with the first two columns corresponding to two words, and the third column representing a human-annotated similarity between the two words.

Assume the file *combined.tab* is located <b>in the same folder as this notebook</b>. You should load this file into a Python dictionary (NOTE: in Python, tuples of strings, i.e. ("tiger","cat") can serve as the keys of dictionaries). This dataset contains many rare words: we need to filter this dataset in order for it to be better suited to the resources we will use in this assignment. So your first goal is to filter this dataset to generate a smaller test set where you will evaluate your word similarity methods.

The first filtering is based on document frequencies in the Brown corpus, in order to remove rare words. In this assignment, we will be treating the <i>paragraphs</i> of the Brown corpus as our "documents", you can iterate over them by using the `paras` method of the corpus reader. You should start by creating a Python list where each element of the list is a set containing the word <b>types</b> from a different paragraph of the Brown corpus: the words should be lower-cased and lemmatized before they are added to the set (keep it around, because you will need this list again later on). Then, using the information in this corpus, calculate document frequencies and remove from your test set any word pairs where at least one of the two words has a document frequency of less than 10 in this corpus. 

The second filtering is based on words with highly ambiguous senses and involves using the NLTK interface to WordNet. Here, you should remove any words which do not have a *single primary sense*. We define single primary sense here as either having only one sense (i.e. only one synset), or where the count (as provided by the WordNet `count()` method for the lemmas associated with a synset) of the most common sense is at least five and at least five times larger than the next most common sense. Also, you should remove any words where the primary sense is not a noun (this information is also in the synset). Store the synset corresponding to this primary sense in a dictionary for use in the next section. Given this definition, remove any word pairs from the test set where at least one of the words does not contain a single primary sense or if the single primary sense is not a noun.

When you have applied these two filtering steps, print out all the pairs in your filtered test set (if you have done this correctly, the total should be more than 10, but less than 50).

(1.5 marks)

In [1]:
import nltk,operator
nltk.download("brown")
from nltk.corpus import brown
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma
import re
def isWord(word):
    result=re.match(r'[a-zA-Z]+',word)
    if result is None:
        return False
    else:
        return True
    
docs=[]
for para in brown.paras():
	docs.append(para)


doc_set_list=[]
doc_freq={}
for para in docs:
    doc_set=set()
    for sentence in para:
        for word in sentence:
            if isWord(word):
                word=word.lower()
                word=lemmatize(word)
                doc_set.add(word)
    doc_set_list.append(doc_set)
for item in doc_set_list:
    for word in item:
        doc_freq[word]=doc_freq.get(word,0)+1

words_file=open('combined.tab','r',encoding="utf8")
line_num=0;
word_similarity=[]
for line in words_file:
    if line_num==0:
        line_num+=1
        continue
    else:
        line_parts=line.split()
        word_similarity.append(line_parts)

def first_filter(words,doc_freq):
    new_words=[]
    for item in words:
        if(doc_freq.get(item[0],0)>=10 and doc_freq.get(item[1],0)>=10):
            new_words.append(item)
    return new_words
word_similarity=first_filter(word_similarity,doc_freq)
from nltk.corpus import wordnet as wn
#this function returns the most common sense and second common sense of synsets
def lemma_count(word,synsets):
    counts={}
    for synset in synsets:
        count=0
        for lemma in synset.lemmas():
            if word ==lemma.name():
                count=count+lemma.count()
        counts[synset.name()]=counts.get(synset.name(),0)+count
    counts=sorted(counts.items(),key=operator.itemgetter(1),reverse= True)
    return counts[0],counts[1]
    

# return a boolean value, if the input word has a singe primary sense,then return True,else return False
def single_sense(word):
    noun_str='.n.'
    synsets=wn.synsets(word)
    if len(synsets)==1 and noun_str in synsets[0].name():
        return True,synsets[0].name()
    most_count,second_count=lemma_count(word,synsets)
    if most_count[1]>=5 and most_count[1]>=5*second_count[1] and noun_str in most_count[0]:
        return True,most_count[0]
    return False,None
    
def second_filter(words):
    word_sense=[]
    word_similarity=[]    
    for item in words:
        word_pair={}
        bool1,sense1=single_sense(item[0])
        bool2,sense2=single_sense(item[1])
        if bool1 and bool2:
            word_pair[item[0]]=sense1
            word_pair[item[1]]=sense2
            word_sense.append(word_pair)
            word_similarity.append(item)
    return word_sense,word_similarity

word_sense,word_similarity=second_filter(word_similarity)
for word in word_similarity:
    print(word)

[nltk_data] Downloading package brown to D:\nltk_data...
[nltk_data]   Package brown is already up-to-date!
['professor', 'doctor', '6.62']
['stock', 'egg', '1.81']
['baby', 'mother', '7.85']
['car', 'automobile', '8.94']
['journey', 'voyage', '9.29']
['coast', 'shore', '9.10']
['brother', 'monk', '6.27']
['journey', 'car', '5.85']
['coast', 'hill', '4.38']
['monk', 'slave', '0.92']
['coast', 'forest', '3.15']
['psychology', 'doctor', '6.42']
['psychology', 'mind', '7.69']
['psychology', 'health', '7.23']
['psychology', 'science', '6.71']
['computer', 'laboratory', '6.78']
['canyon', 'landscape', '7.53']
['century', 'year', '7.59']
['doctor', 'personnel', '5.00']
['school', 'center', '3.44']
['word', 'similarity', '4.75']
['hotel', 'reservation', '8.03']
['type', 'kind', '8.97']
['equipment', 'maker', '5.91']
['luxury', 'car', '6.47']
['soap', 'opera', '7.94']
['planet', 'people', '5.75']


<b>Instructions</b>: Now you will create several dictionaries with similarity scores for pairs of words in your test set derived using the techniques discussed in class. The first of these is the Wu-Palmer scores derived from the hypernym relationships in WordNet, which you should calculate using the primary sense for each word derived above. You can use the built-in method included in the NLTK interface, you don't have to implement your own. When you're done,  print out the Python dictionary of word pair/similarity mappings. 

(0.5 marks)

In [3]:
word_wp_score={}
for word_pair in word_sense:
    word1=list(word_pair.keys())[0]
    word2=list(word_pair.keys())[1]
    word_wp_score[word1,word2]=wn.synset(word_pair[word1]).wup_similarity(wn.synset(word_pair[word2]))
print('------WP_similarity--------------')
word_wpscore_list=[(k,v) for k,v in word_wp_score.items()]
for score in word_wpscore_list:
    print(score)

------WP_similarity--------------
(('similarity', 'word'), 0.3333333333333333)
(('soap', 'opera'), 0.2222222222222222)
(('psychology', 'mind'), 0.5714285714285714)
(('doctor', 'personnel'), 0.13333333333333333)
(('stock', 'egg'), 0.11764705882352941)
(('type', 'kind'), 0.9473684210526315)
(('hill', 'coast'), 0.6666666666666666)
(('psychology', 'health'), 0.21052631578947367)
(('maker', 'equipment'), 0.5)
(('mother', 'baby'), 0.5)
(('century', 'year'), 0.8333333333333334)
(('car', 'journey'), 0.09523809523809523)
(('coast', 'forest'), 0.16666666666666666)
(('doctor', 'professor'), 0.5)
(('psychology', 'science'), 0.9411764705882353)
(('hotel', 'reservation'), 0.375)
(('luxury', 'car'), 0.1111111111111111)
(('doctor', 'psychology'), 0.1111111111111111)
(('center', 'school'), 0.13333333333333333)
(('planet', 'people'), 0.18181818181818182)
(('canyon', 'landscape'), 0.3333333333333333)
(('journey', 'voyage'), 0.8571428571428571)
(('laboratory', 'computer'), 0.35294117647058826)
(('slave', 

**Instructions:** Next, you will calculate Positive PMI (PPMI) for your word pairs using statistics derived from the Brown: you should use the same set up as you did to calculate document frequency above: paragraphs as documents, lemmatized, lower-cased, and with term frequency information removed by conversion to Python sets. You need to use the basic method for calculating PPMI introduced in class (and also in the reading) which is appropriate for any possible definition of co-occurrence (here, appearing in the same paragraph), but you should only calculate PPMI for the words in your test set. You must avoid building the entire co-occurrence matrix, instead you should keeping track of the sums you need for the probabilities as you go along. When you have calculated PMI for all the pairs, your code should print out the Python dictionary of word-pair/PPMI-similarity mappings.

(1 mark)

In [4]:
from math import log
def cal_PPMI(word1,word2):
    sum_x=0
    sum_y=0
    sum_xy=0.0
    doc_len=0
    for para in doc_set_list:
        doc_len=doc_len+len(para)
        if word1 in para:
            sum_x+=1
        if word2 in para:
            sum_y+=1
        if word1 in para and word2 in para:
            sum_xy+=1
    if sum_xy==0:
        return 0
    return log(sum_xy*doc_len/(sum_x*sum_y),2)

word_PMI={}
for pair in word_similarity:
    word_PMI[pair[0],pair[1]]=cal_PPMI(word1=pair[0],word2=pair[1])
print('-----PPMI------------')
word_PMI_list=[(k,v) for k,v in word_PMI.items()]
for pmi in word_PMI_list:    
    print(pmi)

-----PPMI------------
(('soap', 'opera'), 9.68820034480337)
(('psychology', 'mind'), 8.234400594245404)
(('doctor', 'personnel'), 7.63013128854784)
(('stock', 'egg'), 7.177238425525991)
(('type', 'kind'), 6.070724133544106)
(('planet', 'people'), 5.874235965846784)
(('psychology', 'health'), 0)
(('professor', 'doctor'), 0)
(('journey', 'car'), 0)
(('century', 'year'), 6.322216464518382)
(('word', 'similarity'), 0)
(('school', 'center'), 6.09424906085496)
(('baby', 'mother'), 8.566282766033577)
(('coast', 'forest'), 8.449340711878152)
(('coast', 'hill'), 6.611893724664112)
(('psychology', 'science'), 10.54550165864841)
(('hotel', 'reservation'), 8.358051743111039)
(('luxury', 'car'), 7.739332554013687)
(('canyon', 'landscape'), 0)
(('coast', 'shore'), 10.029580802356906)
(('journey', 'voyage'), 0)
(('equipment', 'maker'), 9.750317934731225)
(('monk', 'slave'), 0)
(('psychology', 'doctor'), 8.974085689765202)
(('computer', 'laboratory'), 0)
(('brother', 'monk'), 8.366272249916008)
(('car

**Instructions:** Next, you will derive similarity scores using the LSA method, i.e. apply SVD and truncate to get a dense vector and then use cosine similarity between the two vectors for each word pair. You can use the Distributed Semantics notebook as a starting point, but note that since you are interested here in word semantics, you will be constructing a matrix where the (non-sparse) rows correspond to words in the vocabulary, and the (sparse) columns correspond to the texts where they appear (this is the opposite of the notebook). Again, use the Brown corpus, in the same format as with PMI and document frequency. After you have a matrix in the correct format, use truncatedSVD in Sci-kit learn to produce dense vectors of length 500, and then use cosine similarity to produce similarities for your word pairs. Print out the corresponding Python dictionary.

(1 mark)

In [5]:
from sklearn.feature_extraction import DictVectorizer
def get_BOW(word_set):
    BOW={}
    for word in word_set:
        BOW[word]=BOW.get(word,0)+1
    return BOW
doc_BOW=[]
for para in doc_set_list:
    doc_BOW.append(get_BOW(para))
vectorizer = DictVectorizer()
brown_matrix = vectorizer.fit_transform(doc_BOW).transpose()
word_list=vectorizer.get_feature_names()
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=500)
brown_matrix = svd.fit_transform(brown_matrix)

from scipy.spatial.distance import cosine as cos_distance
def cal_cosine(word1,word2):
    index1=word_list.index(word1)
    index2=word_list.index(word2)
    vect1=brown_matrix[index1]
    vect2=brown_matrix[index2]
    return 1-cos_distance(vect1,vect2)
word_cos={}
for pair in word_similarity:
    word_cos[pair[0],pair[1]]=cal_cosine(pair[0],pair[1])
print('------cosine similarity---------')
word_cos_list=[(k,v) for k,v in word_cos.items()]
for cos in word_cos_list:    
    print(cos)

------cosine similarity---------
(('soap', 'opera'), 0.00025964372628661803)
(('psychology', 'mind'), 0.12813886007943509)
(('doctor', 'personnel'), 0.074921573022329158)
(('stock', 'egg'), 0.13346460249343828)
(('type', 'kind'), 0.027548256088371748)
(('planet', 'people'), 0.035719182555899454)
(('psychology', 'health'), 0.031756471135822473)
(('professor', 'doctor'), 0.060523962932479525)
(('journey', 'car'), 0.0044085526717014822)
(('century', 'year'), 0.070397957995586014)
(('word', 'similarity'), -0.0022969287162977192)
(('school', 'center'), 0.044219865934668756)
(('baby', 'mother'), 0.32932135237253335)
(('coast', 'forest'), 0.14955707541810959)
(('coast', 'hill'), 0.23082207163089419)
(('psychology', 'science'), 0.25243848713189676)
(('hotel', 'reservation'), 0.088228424951277473)
(('luxury', 'car'), 0.072459673138526082)
(('canyon', 'landscape'), 0.10818575989492718)
(('coast', 'shore'), 0.39518445564382532)
(('journey', 'voyage'), 0.10080315413157059)
(('equipment', 'maker'),

**Instructions:** Next, you will derive a similarity score from word2vec vectors, using the Gensim interface. Check the Gensim word2vec tutorial for details on the API: https://radimrehurek.com/gensim/models/word2vec.html. Again, you should use the Brown for this, but for word2vec you don't need to worry about paragraphs: feel free to train your model at the sentence level instead. Your vectors should have the same number of dimensions as LSA (500), and you need to run for 50 iterations. This may take a while (several minutes), but that's okay, you won't be marked based on the speed of this. You should extract the similarites you need directly from the Gensim model, put them in a Python dictionary, and print them out.

(0.5 mark)

In [6]:
from gensim.models import Word2Vec
sents=brown.sents()
model = Word2Vec(sents, size=500, window=6, min_count=5, workers=4,iter=50)
word_gensim={}
for pair in word_similarity:
    word_gensim[pair[0],pair[1]]=model.wv.similarity(pair[0],pair[1])
print('------gensim_similarity---------')
word_gensim_list=[(k,v) for k,v in word_gensim.items()]
for line in word_gensim_list:    
    print(line)

C:\Users\zsl\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


------gensim_similarity---------
(('soap', 'opera'), -0.0088188472188100997)
(('psychology', 'mind'), 0.039572662774232145)
(('doctor', 'personnel'), -0.097432954302824809)
(('stock', 'egg'), 0.14654004902004897)
(('type', 'kind'), 0.28293432515256745)
(('planet', 'people'), -0.021094552976027192)
(('psychology', 'health'), 0.1984679528379848)
(('professor', 'doctor'), 0.10186272540240561)
(('journey', 'car'), 0.19510848962704364)
(('century', 'year'), 0.26906763797736022)
(('word', 'similarity'), 0.042937133433088963)
(('school', 'center'), -0.042526015157573427)
(('baby', 'mother'), 0.21179388750586547)
(('coast', 'forest'), 0.24821613388580582)
(('coast', 'hill'), 0.43814520923570377)
(('psychology', 'science'), 0.34627418639805718)
(('hotel', 'reservation'), 0.069781898075272936)
(('luxury', 'car'), 0.17803111841588426)
(('canyon', 'landscape'), 0.18487182783360856)
(('coast', 'shore'), 0.41827670293537644)
(('journey', 'voyage'), 0.47153198406543079)
(('equipment', 'maker'), 0.222


**Instructions:** Finally, you should compare all the similarities you've created to the gold standard you loaded and filtered in the first step. For this, you can use the Pearson correlation co-efficient (`pearsonr`), which is included in scipy (`scipy.stats`). Be careful converting your dictionaries to lists for this purpose, the data for the two datasets needs to be in the same order for correct comparison using correlation. Write a general function, then apply it to each of the similarity score dictionaries, and print out the result for each (be sure to label them!). Hint: All of the methods used here should be markedly above 0, but also far from 1 (perfect correlation); if you're not getting reasonable results, go back and check your code for bugs!  

(0.5 mark)


In [7]:
from scipy import stats
def norm_list(aList):
    new_list=[]
    for line in aList:
        list_parts=[]
        list_temp=list(line[0])
        list_temp.sort()
        list_parts.append(list_temp)
        list_parts.append(line[1])
        new_list.append(list_parts)
    new_list=sorted(new_list, key=lambda x:x[0][0])
    final_list=[]
    for line in new_list:
        final_list.append(line[1])
    return final_list
test_list=[]
for word in word_similarity:
    line_part=[word[0],word[1]]
    line_parts=[line_part,float(word[2])]
    test_list.append(line_parts)
test_list=norm_list(test_list)
word_gensim_list=norm_list(word_gensim_list)
word_cos_list=norm_list(word_cos_list)
word_wpscore_list=norm_list(word_wpscore_list)
word_PMI_list=norm_list(word_PMI_list)
print()
print('-------golden standard vs Wu_Palmer Similarity-------')
print(stats.pearsonr(test_list,word_wpscore_list)[0])
print()
print('-------golden standard vs PPMI-------')
print(stats.pearsonr(test_list,word_PMI_list)[0])
print()
print('-------golden standard vs cosine similarity-------')
print(stats.pearsonr(test_list,word_cos_list)[0])
print()
print('-------golden standard vs gensim-------')
print(stats.pearsonr(test_list,word_gensim_list)[0])


-------golden standard vs Wu_Palmer Similarity-------
0.333937557663

-------golden standard vs PPMI-------
0.089410098586

-------golden standard vs cosine similarity-------
0.0554687060471

-------golden standard vs gensim-------
0.194608882863


## Bonus: Improving the correlation

**Instructions:** To get the extra credit, you should try to get a noticeably (at least 0.05) higher correlation than that of the highest correlation that you found in the main assignment. To accomplish this, you can use any resources included or accessible via NLTK EXCEPT the Google word2vec vectors. Half of your mark here will be based on the thought and effort involved in your solution (if you tried something sensible that didn't work, show us), the other half will be based on the actual performance increase, which you should demonstrate. For full credit, be sure to explain exactly what you are doing.

(1 bonus mark) 

## A final word

Normally, we would not use a corpus as small as the Brown for the purposes of building distributional word vectors. Also, note that filtering our test set to just words we are likely to do well on would typically be considered cheating.